# Mesurer et afficher température, pression, hygrométrie, température Haute Résolution et éclairement

Utilisation des capteurs 
 * Press-Hygro-Temp : Adafruit BME280
 * Température HR   : Adafruit MCP9808
 * Luxmètre         : Adafruit TSL2561
 * LCD 2x16         : Adafruit JHD1313M1
 
Il est souhaitable d'aborder cette démonstration _après_ avoir effectué la manipulation avec le capteur BME280 seul.

Activité réalisée avec une raspberry Pi3 fonctionnant avec une image Debian Stretch/Buster fournie par l'IFÉ ENS de Lyon, disposant des librairies INTEL mraa et upm. en utilisant le bus i2c de la raspberry.

Sur le site de Adafruit https://learn.adafruit.com/adafruit-bme280-humidity-barometric-pressure-temperature-sensor-breakout/pinouts on lit que 4 broches doivent être connectées au raspberryPi.
 * Vin - cest la broche d'alimentation : elle peut être reliée à 3,3V ou 5V (Nous la relierons au 3,3V dans cet exercice). 
 * GND - broche à relier à une broche ground du RasberryPi
 * SCK - c'est la broche d'horloge du bus I2C à relier à la broche 5 (SCL)
 * SDI - c'est la broche de données du bus I2C à relier à la broche 3 (SDA)

De même pour les capteurs MCP9808 et TSL2561 on dispose des broches
 * Vdd - cest la broche d'alimentation : elle peut être reliée à 3,3V ou 5V (Nous la relierons au 3,3V dans cet exercice). 
 * GND - broche à relier à une broche ground du RasberryPi
 * SCL - c'est la broche d'horloge du bus I2C à relier à la broche 5 (SCL)
 * SDA - c'est la broche de données du bus I2C à relier à la broche 3 (SDA)

<span style='color:red'>ATTENTION : des informations contradictoires sont disponibles sur internet sur la nécessité d'utiliser (ou pas) un [convertisseur de tension (levelshifter)](https://www.adafruit.com/product/757) (voltage level shifter). Par précaution  nous utilisons ce composant qui évite que les broches SCL et SDA soient directement reliées à la raspberry lorsque les capteurs sont alimentés sous une tension de 5V. La raspberry étant conçue pour recevoir au maximum en entrée une tension de 3,3V c'est cette tension qui est appliquée sur la sortie du convertisseur. Ainsi les voies SCL et SDA sont ajustées à cette valeur alors que l'entrée se fait en 5V fournis au(x) capteur(s). </span> 

Figure du montage :
![Montage ministation météo](../StationMeteo/images/bus_ministation_schema_bb.png)
![Licence Charles-Henri Eyraud](../StationMeteo/images/Licence.jpg)

Photo du Montage :
![Montage ministation météo](../StationMeteo/images/bus_ministation_photo.jpg)
![Licence Charles-Henri Eyraud](../StationMeteo/images/Licence.jpg)


Les mesures sont efectuées en boucle (infinie ou limitée) et l'affichege est cyclique après chaque mesure.

In [5]:
#Importation des bibliothèques du projet et simplification du nom des fonctions utilisées
import mraa, math, time
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm
from upm import pyupm_mcp9808 as mcp
from upm import pyupm_tsl2561 as tsl
# Utilisation de variables pour le numéro du bus i2c, l'adresse des composants, la durée d'allumage...
bus = 0
bmeAddr = 0x77
lcdAddress = 0x3E
rgbAddress = 0x62
mcpAddr = 0x18
tslAddr = 0x39
# Paramètres de configuration du capteur température de précision (voir documentation)
UPPER_TEMP = 0x02
LOWER_TEMP = 0x03
CRIT_TEMP = 0x04
# Autres variables
duration = 10
charDeg = 0
charDelta = 1
#Couleur du fond [r,g,b]
color = [192,32,64]
#Dessin du caractère degré
degree = (
    0b01100,
    0b10010,
    0b01100,
    0b00000,
    0b00000,
    0b00000,
    0b00000,
    0b00000)
#Dessin du caractère delta
delta = (
    0b00000,
    0b00000,
    0b00000,
    0b00100,
    0b01110,
    0b11011,
    0b10011,
    0b11111)
#Altitude locale 
localAlt = 169 #Altitude de Lyon
#Déclaraton et nommage des composants utilisés
bme = bmp.BME280(bus,bmeAddr)
lcd = jdm.Jhd1313m1(bus,lcdAddress,rgbAddress)
temp = mcp.MCP9808(bus,mcpAddr)
luxtsl = tsl.TSL2561(bus,tslAddr)
#Initialisation du capteur avant la mesure
bme.update()
#Création du caractère degré
lcd.createChar(charDeg,degree)
#Création du caractère delta
lcd.createChar(charDelta,delta)
# Fonctions de mesure
# mesures via le BME280
def mes_bmTPA(thisSensor) :
    thisSensor.update ()
    bmTemp = thisSensor.getTemperature ()
    bmPress = convertSeaLevel (thisSensor.getPressure () /100.0)
    bmHum = thisSensor.getHumidity ()
    return(bmTemp, bmPress, bmHum)
# Mesures via le MCP9808
def mes_mcpTemp(thisSensor):
    mcpTemp = thisSensor.getTemp ()
    return (mcpTemp)
# Mesures via le TSL2561
def mes_tslLux(thisSensor) :
    lux = thisSensor.getLux ()
    return(lux)
#Fonction de calcul de la pression au niveau de la mer
def convertSeaLevel (m_Pressure):
    sl_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
    return sl_Pressure
#
# Boucle du programme
# while True donne une boucle infinie 
# for in range() fournit un nombre choisi d'itérations
# l'indice i peut-être utilisé pour contrôler les couleurs
#
#while True :
for i in range(0,4,1) :
    #Rotation des couleurs sur 3 valeurs pour signaler une nouvelle mesure
    color = [color[1],color[2],color[0]]
    #Mesure de la température de la pression de l'hygrométrie
    [tempBME,pressBME,hygroBME] = mes_bmTPA(bme)
    tempHR = mes_mcpTemp(temp)
    lux = mes_tslLux(luxtsl)
    #Chargement de la couleur
    lcd.setColor(color[0],color[1],color[2])
    # bloc 1 ==================================================
    #Positionnement du curseur sur l'écran ligne du haut case 0
    lcd.setCursor(2, 0)
    #Affichage de la température BME sur l'écran
    lcd.write("T. : {0:7.2f}  ".format(tempBME))
    lcd.write(chr(charDeg))
    lcd.write("C  ")
    #Positionnement du curseur sur l'écran ligne du haut case 0
    lcd.setCursor(1, 0)
    #Affichage de la température MCP sur l'écran
    lcd.write("T. : {0:7.2f}  ".format(tempHR))
    lcd.write(chr(charDeg))
    lcd.write("C  ")
    time.sleep(3)    
    # bloc 2 ==================================================
    #Positionnement du curseur sur l'écran ligne du haut case 0
    lcd.setCursor(2, 0)
    #Affichage de la pression BME sur l'écran
    lcd.write("P. : {0:7.2f} hPa".format(pressBME))
    #Positionnement du curseur sur l'écran liigne du bas case 0
    lcd.setCursor(1, 0)
    #Affichage de l'hygrométire sur l'écran
    status = lcd.write("H. : {0:7.2f}   %".format(hygroBME))
    time.sleep(3)
    # bloc 3 ==================================================
    #Positionnement du curseur sur l'écran ligne du haut case 0
    lcd.setCursor(2, 0)
    #Affichage de la luminosité sur l'écran
    lcd.write("L. : {0:7.2f}  lux".format(lux))    
    lcd.setCursor(1, 0)
    #Affichage de la différence de température sur l'écran
    lcd.write(chr(charDelta))
    status = lcd.write("T : {0:7.2f}  ".format(tempBME - tempHR))
    lcd.write(chr(charDeg))
    lcd.write("C  ")
    time.sleep(3)
    status = lcd.clear()
    time.sleep(1)
status = lcd.clear()
status = lcd.backlightOff()